# Spotify API

<img width="1000" src = "https://www.markhneedham.com/blog//uploads/2020/01/spotify-logo.png" />

- Spotify é um serviço digital que dá acesso instantâneo a milhões de músicas, podcasts, vídeos e outros conteúdos de criadores no mundo todo.

- É liberado o acesso às suas APIs para que os desenvolvedores possam explorar seus dados.

- [Url para os endpoints do Spotify](https://developer.spotify.com/) 





## Libraries Import

In [41]:
import os
import json
import base64
import pandas as pd
import lyricsgenius as lg
from requests import post, get
from dotenv import load_dotenv

#### Load personal keys

In [42]:
load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

genius = lg.Genius(os.getenv("GENIUS_TOKEN"), skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)

## Making requests to Spotify API

#### Function to get API's access token

In [43]:
def get_token():
    auth_string = CLIENT_ID + ":" + CLIENT_SECRET
    auth_string_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_string_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded",
    }

    data = {"grant_type": "client_credentials"}
    response = post(url, headers=headers, data=data)
    json_response = json.loads(response.content)
    token = json_response.get("access_token")

    return token

In [44]:
def default_headers(token):
    return {"Authorization": "Bearer " + token}


In [45]:
token = get_token()

#### Search for an artist

In [46]:
def get_artist(token, artist):
    url = "https://api.spotify.com/v1/search"
    headers = default_headers(token)
    query = f"?q={artist}&type=artist&limit=1"

    url_with_params = url + query
    response = get(url_with_params, headers=headers)

    json_reponse = json.loads(response.content)["artists"]["items"]
    return json_reponse

<img width = "400" src = "https://cdn.worldvectorlogo.com/logos/ac-dc-1.svg"/>

In [47]:
artist_infos = get_artist(token, "AC/DC")
artist_infos

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/711MCceyCBcFnzjGY4Q7Un'},
  'followers': {'href': None, 'total': 29485534},
  'genres': ['australian rock', 'hard rock', 'rock'],
  'href': 'https://api.spotify.com/v1/artists/711MCceyCBcFnzjGY4Q7Un',
  'id': '711MCceyCBcFnzjGY4Q7Un',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab6761610000e5ebc4c77549095c86acb4e77b37',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/ab67616100005174c4c77549095c86acb4e77b37',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/ab6761610000f178c4c77549095c86acb4e77b37',
    'width': 160}],
  'name': 'AC/DC',
  'popularity': 83,
  'type': 'artist',
  'uri': 'spotify:artist:711MCceyCBcFnzjGY4Q7Un'}]

In [48]:
artist_id = artist_infos[0]["id"]
artist_id

'711MCceyCBcFnzjGY4Q7Un'

#### Get top tracks of selected artist

In [49]:
def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = default_headers(token)

    response = get(url, headers=headers)
    json_response = json.loads(response.content)["tracks"]
    return json_response

In [50]:
songs = get_songs_by_artist(token, artist_id)
songs

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/711MCceyCBcFnzjGY4Q7Un'},
     'href': 'https://api.spotify.com/v1/artists/711MCceyCBcFnzjGY4Q7Un',
     'id': '711MCceyCBcFnzjGY4Q7Un',
     'name': 'AC/DC',
     'type': 'artist',
     'uri': 'spotify:artist:711MCceyCBcFnzjGY4Q7Un'}],
   'available_markets': ['AR',
    'AU',
    'AT',
    'BE',
    'BO',
    'BR',
    'BG',
    'CA',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DK',
    'DO',
    'DE',
    'EC',
    'EE',
    'SV',
    'FI',
    'FR',
    'GR',
    'GT',
    'HN',
    'HK',
    'HU',
    'IS',
    'IE',
    'IT',
    'LV',
    'LT',
    'LU',
    'MY',
    'MT',
    'MX',
    'NL',
    'NZ',
    'NI',
    'NO',
    'PA',
    'PY',
    'PE',
    'PH',
    'PL',
    'PT',
    'SG',
    'SK',
    'ES',
    'SE',
    'CH',
    'TW',
    'TR',
    'UY',
    'US',
    'GB',
    'AD',
    'LI',
    'MC',
    'ID',
    'JP',
    'TH',
    'VN',
    'RO',

<img src="https://routenote.com/blog/wp-content/uploads/2019/12/Genius-lyrics.jpeg" />

In [51]:
songs_details = []
artist = genius.search_artist("AC/DC", max_songs=1)

for song in songs:
    song_lyrics = artist.song(song["name"]).lyrics

    songs_details.append({
        "name": song["name"],
        "artist": "AC/DC",
        "type": song["album"]["album_type"],
        "url": song["external_urls"]["spotify"],
        "popularity": song["popularity"],
        "duration (ms)": song["duration_ms"],
        "release_date": song["album"]["release_date"],
        "lyrics": song_lyrics
                })

Searching for songs by AC/DC...

Song 1: "T.N.T."

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "You Shook Me All Night Long" by AC/DC...
Done.
Searching for "Back In Black" by AC/DC...
Done.
Searching for "Thunderstruck" by AC/DC...
Done.
Searching for "Highway to Hell" by AC/DC...
Done.
Searching for "Hells Bells" by AC/DC...
Done.
Searching for "Shoot to Thrill" by AC/DC...
Done.
Searching for "Dirty Deeds Done Dirt Cheap" by AC/DC...
Done.
Searching for "It's a Long Way to the Top (If You Wanna Rock 'N' Roll)" by AC/DC...
Done.
Searching for "Rock N Roll Train" by AC/DC...
Done.


In [52]:
songs_details

[{'name': 'You Shook Me All Night Long',
  'artist': 'AC/DC',
  'type': 'album',
  'url': 'https://open.spotify.com/track/2SiXAy7TuUkycRVbbWDEpo',
  'popularity': 85,
  'duration (ms)': 210173,
  'release_date': '1980-07-25',
  'lyrics': "53 ContributorsYou Shook Me All Night Long Lyrics\nShe was a fast machine, she kept her motor clean\nShe was the best damn woman that I ever seen\nShe had the sightless eyes, telling me no lies\nKnocking me out with those American thighs\nTaking more than her share, had me fighting for air\nShe told me to come, but I was already there\n'Cause the walls start shaking, the Earth was quaking\nMy mind was aching, and we were making it\n\nAnd you shook me all night long\nYeah, you shook me all night long\n\nWorking double-time on the seduction line\nShe's one of a kind, she's just a-mine all mine\nWanted no applause, just another course\nMade a meal outta me, and come back for more\nHad to cool me down to take another round\nNow, I'm back in the ring to ta

In [53]:
songs_df = pd.DataFrame(songs_details).sort_values(by=["popularity"], ascending=False).reset_index(drop=True)
songs_df

,name,artist,type,url,popularity,duration (ms),release_date,lyrics
0,You Shook Me All Night Long,AC/DC,album,https://open.spotify.com/track/2SiXAy7TuUkycRV...,85,210173,1980-07-25,53 ContributorsYou Shook Me All Night Long Lyr...
1,Back In Black,AC/DC,album,https://open.spotify.com/track/08mG3Y1vljYA6bv...,84,255493,1980-07-25,72 ContributorsTranslationsPortuguêsBack in Bl...
2,Thunderstruck,AC/DC,album,https://open.spotify.com/track/57bgtoPSgt236Hz...,84,292880,1990-09-24,70 ContributorsTranslationsDeutschThunderstruc...
3,Highway to Hell,AC/DC,album,https://open.spotify.com/track/2zYzyRzz6pRmhPz...,83,208400,1979-07-27,100 ContributorsTranslationsDeutschHighway to ...
4,T.N.T.,AC/DC,album,https://open.spotify.com/track/7LRMbd3LEoV5wZJ...,78,214666,1976-05-14,45 ContributorsT.N.T. Lyrics\nOi! Oi! Oi! Oi! ...
5,Hells Bells,AC/DC,album,https://open.spotify.com/track/69QHm3pustz01CJ...,75,312293,1980-07-25,49 ContributorsHells Bells Lyrics\n\nI'm a rol...
6,Shoot to Thrill,AC/DC,album,https://open.spotify.com/track/0C80GCp0mMuBzLf...,73,317426,1980-07-25,26 ContributorsShoot to Thrill Lyrics\nAll you...
7,Dirty Deeds Done Dirt Cheap,AC/DC,album,https://open.spotify.com/track/2d4e45fmUnguxh6...,73,231933,1976-09-20,81 ContributorsDirty Deeds Done Dirt Cheap Lyr...
8,It's a Long Way to the Top (If You Wanna Rock ...,AC/DC,album,https://open.spotify.com/track/7nemcVsXVFZF01i...,71,301226,1976-05-14,32 ContributorsIt’s a Long Way to the Top (If ...
9,Rock N Roll Train,AC/DC,album,https://open.spotify.com/track/4feXJ5IC3avXsVW...,68,261506,2008-10-20,11 ContributorsRock N’ Roll Train Lyrics\nOne ...


In [54]:
print(songs_df["lyrics"][4])

45 ContributorsT.N.T. Lyrics
Oi! Oi! Oi! Oi! Oi! Oi!
Oi! Oi! Oi! Oi! Oi! Oi!
Oi! Oi! Oi!

See me ride out of the sunset
On your color TV screen
Out for all that I can get
If you know what I mean

Women to the left of me
And women to the right
Ain't got no gun
Ain't got no knife
Don't you start no fight

'Cause I'm T.N.T., I'm dynamite
T.N.T., and I'll win the fight
T.N.T., I'm a power load
T.N.T., watch me explode

I'm dirty, mean, I'm mighty unclean
I'm a wanted man
Public enemy number one
Understand?
You might also like
So lock up your daughter
Lock up your wife
Lock up your back door
Run for your life
The man is back in town
So don't you mess me 'round

'Cause I'm T.N.T., I'm dynamite
T.N.T., and I'll win the fight
T.N.T., I'm a power load
T.N.T., watch me explode


T.N.T
Oi! Oi! Oi!
T.N.T
Oi! Oi! Oi!
T.N.T
Oi! Oi! Oi!
T.N.T
Oi! Oi! Oi!
T.N.T (Oi! Oi! Oi!)
I'm dynamite
T.N.T (Oi! Oi! Oi!)
And I'll win the fight
T.N.T (Oi! Oi! Oi!)
I'm a power load
T.N.T., watch me explode!32Embed


## Export Data

In [56]:
songs_df.to_csv("../data/songs.csv", index=None)